In [1]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import cftime
import datetime
%matplotlib inline
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from sklearn.metrics.pairwise import haversine_distances

grid = xr.open_dataset(
    '/glade/work/vcooper/grid_ref/sithick_SImon_CESM2_piControl_r1i1p1f1_gn_110001-120012.nc')

/glade/u/home/vcooper/miniconda3/envs/camus/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'sithick' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,


## Load Data
### Obs: BGEP moorings, SWIFT buoys, SODA (APL)
### Bootstrap obs (satellite)
### Wave-ice model simulation

In [6]:
# Read in mooring data for 4 time periods from 3 files
bdir = '/glade/work/lettier/BGEP/'
files = ['BGEP-A_AWAC_2012-2013_converted.nc',
         'BGEP-D_AWAC_2014-2015_converted.nc',
         'BGEP-A_AWAC_2017-2018_converted.nc',]        
bounds = [['2012-08','2012-11'],
          ['2015-08','2015-10'],
          ['2017-10','2017-12'],
          ['2018-07','2018-10']]

bds12 = xr.open_dataset(bdir+files[0]).sel(time=slice(bounds[0][0],bounds[0][1]))
bds15 = xr.open_dataset(bdir+files[1]).sel(time=slice(bounds[1][0],bounds[1][1]))
bds17 = xr.open_dataset(bdir+files[2]).sel(time=slice(bounds[2][0],bounds[2][1]))
bds18 = xr.open_dataset(bdir+files[2]).sel(time=slice(bounds[3][0],bounds[3][1]))

# model data
mdir = '/glade/work/lettier/CICE/cicefsdww3i_output_concat/' ## location of file before changing dim names
mdir_adj = '/glade/u/home/vcooper/work/BGEP_vtc/adjfiles/' # location of adjusted files, changed dims to lat and lon
ice_file = 'cicefsdww3i.cice.h.concat.0068-0084.nc' # monthly ice output
wave_file = 'cicefsdww3i.ww3.sigheight.2000-2016.nc' # daily wave output
mwds = xr.open_dataset(mdir_adj+wave_file)
mwds = mwds.set_coords(['longitude','latitude'])

ww18 = xr.open_dataset('/glade/work/vcooper/BGEP_vtc/ww3.2018.nc') # 2018 model output

cice18 = xr.open_dataset('/glade/work/vcooper/BGEP_vtc/cicefsdww3i.cice.h1.2018.nc') # 2018 model output
start = pd.Timestamp(ww18.time.values[0])
end = pd.Timestamp(ww18.time.values[-1])
t = np.linspace(start.value, end.value, len(cice18.time))
t = (pd.to_datetime(t)).round('D')
cice18.time.values = np.asarray(t) # adjust time to be year 2018

# SODA data
sodaA = xr.open_dataset('/glade/work/vcooper/BGEP_vtc/SODA/SODA_A_sigWaves.mat_converted.nc')
sodaB = xr.open_dataset('/glade/work/vcooper/BGEP_vtc/SODA/SODA_B_sigWaves.mat_converted.nc')
sodaC = xr.open_dataset('/glade/work/vcooper/BGEP_vtc/SODA/SODA_C_sigWaves.mat_converted.nc')


# swift data
swiftpath = '/glade/work/lettier/SWIFT_SeaState2015/'
swift_files = ('SWIFT13_11-14Oct2015_30min_converted.nc','SWIFT11_31Oct-01Nov2015_30min_converted.nc',
              'SWIFT09_06-08Oct2015_30min_converted.nc','SWIFT15_23-25Oct2015_30min_converted.nc',
              'SWIFT12_02Oct2015_30min_converted.nc','SWIFT11_23-24Oct2015_30min_converted.nc',
              'SWIFT09_11-14Oct2015_30min_converted.nc','SWIFT11_10Oct2015_30min_converted.nc',
              'SWIFT14_23-24Oct2015_30min_converted.nc','SWIFT09_31Oct-01Nov2015_30min_converted.nc',
              'SWIFT12_23-24Oct2015_30min_converted.nc','SWIFT15_11-13Oct2015_30min_converted.nc',
              'SWIFT14_10Oct2015_30min_converted.nc','SWIFT09_02Nov2015_30min_converted.nc',
              'SWIFT14_04Oct2015_30min_converted.nc','SWIFT13_31Oct-01Nov2015_30min_converted.nc',
              'SWIFT14_11-13Oct2015_30min_converted.nc','SWIFT13_23-24Oct2015_30min_converted.nc',
              'SWIFT14_16-18Oct2015_30min_converted.nc','SWIFT15_31Oct-01Nov2015_30min_converted.nc',
              'SWIFT11_16-18Oct2015_30min_converted.nc','SWIFT11_11-14Oct2015_30min_converted.nc',
              'SWIFT09_23-24Oct2015_30min_converted.nc','SWIFT11_04Oct2015_30min_converted.nc',
              'SWIFT12_11-14Oct2015_30min_converted.nc','SWIFT12_04Oct2015_30min_converted.nc',
              'SWIFT12_06-08Oct2015_30min_converted.nc')

swift_files = sorted(swift_files)  # sorted list of all SWIFT files
swift_names = swift_files # initialize list of shortened names for use as xarray data
swift_dict = {}
nswift = len(swift_names)

for i, file in enumerate(swift_files):
    swift_names[i] = file[:-19] # drops final 19 chars
    swift_dict[swift_names[i]] = xr.open_dataset(swiftpath + file) # datasets for each file, key to swift_names

    
# Get observations from NSIDC bootstrap, regridded to match CICE grid
path = '/glade/work/vcooper/BGEP_vtc/regrid_sat/' ## location of regridded file
ice_file = 'seaice_conc_daily_nh_1979-2018_cicegrid.nc' # name of regridded file
boot = xr.open_dataset(path+ice_file)


### Function to calculate distance to ice edge

In [7]:
def icedistance(iceconc_input):
    # turn icefracs to numpy array
    icefracsnp = iceconc_input.values
    lats = iceconc_input.latitude.values
    lons = iceconc_input.longitude.values


    # create array to hold the distances
    distances = icefracsnp.copy() # same size array as the evaluated data
    distances -= distances # make zeros or nan; we will keep these values for cells that don't need a calc

    
    ##### GET OPEN WATER -> WATER/ICE EDGE LOCATIONS #####
    
    # get all open water locations except at edge of domain to avoid computation breaking
    icefracsnp_noborder = icefracsnp[1:-1,1:-1] # exclude borders for open water checking neighbors
    locations_openw = np.transpose(np.where(icefracsnp_noborder<0.001))
    locations_openw += 1 # adjust indices for the border exclusion

    # create 4 arrays, each represents the offset of open water location in coords by 1 unit
    latp1 = np.append(locations_openw[:,0]+1,locations_openw[:,1]).reshape(locations_openw.shape,order='F')
    latm1 = np.append(locations_openw[:,0]-1,locations_openw[:,1]).reshape(locations_openw.shape,order='F')
    lonp1 = np.append(locations_openw[:,0],locations_openw[:,1]+1).reshape(locations_openw.shape,order='F')
    lonm1 = np.append(locations_openw[:,0],locations_openw[:,1]-1).reshape(locations_openw.shape,order='F')

    # sum the icefracs of 4 neighbor cells at each open water cell
    iceneighborsum = np.nansum(np.stack((icefracsnp[latp1[:,0],latp1[:,1]],
                                         icefracsnp[lonm1[:,0],lonm1[:,1]],
                                         icefracsnp[lonp1[:,0],lonp1[:,1]],
                                         icefracsnp[latm1[:,0],latm1[:,1]])),axis=0)

    # get index of the open water cells with non-zero ice neighbor # these are values for which we will calc distance
    wateredge = locations_openw[np.where(iceneighborsum>0)]
    wateredgeT = wateredge.T
    wateredgelatlon = np.array([[lats[wateredgeT[0],wateredgeT[1]]],
                                [lons[wateredgeT[0],wateredgeT[1]]]]).squeeze().T # Nx2 matrix of lat,lon
    
    ##### CALCULATION OF DISTANCES #####
    
    # get all cell locations with ice
    icewhere = np.where(icefracsnp>0.001)
    icecells = np.transpose(icewhere) # index by array position
    icelatlon = np.array([[lats[icewhere]],
                          [lons[icewhere]]]).squeeze().T # Nx2 matrix of lat,lon

    # calculate minimum distance
    mindist = haversine_distances(np.deg2rad(icelatlon),
                                  np.deg2rad(wateredgelatlon)).min(axis=1)*6371000/1000 # x by Radius-earth for km
    
    icecellsT = np.transpose(icecells) # transpose for vectorized indexing
    distances[icecellsT[0],icecellsT[1]] = mindist # put mindist into each grid point
    
#     consider adding something like this and removing the loop? so it outputs xarray
#     dist_series = iceconc_input.copy()
#     dist_series.values = np.array(disttest_v04)


    return(distances)

### Interpolate cice18 aice to have 6h timesteps

In [9]:
cice18_6h = cice18.aice_d.interp(time=ww18.time.values)

## Find a 2018 wave event

In [11]:
ww18.sel(time='2018-07')[0].where(cice18_6h.sel)

KeyError: 0